In [1]:
import calendar
import gc
import os
import sys
import json

import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
import scipy
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from implicit.als import AlternatingLeastSquares
import zipfile


/home/mle-user/mle_projects/mle-pr-final/env_recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('/home/mle-user/mle_projects/mle-pr-final/')
from src.helpers import start_mlflow_server, load_config

In [3]:
os.getcwd()

'/home/mle-user/mle_projects/mle-pr-final/notebooks'

In [4]:
config = load_config('/home/mle-user/mle_projects/mle-pr-final/config/config.json')

In [5]:
config

{'experiment_name': 'mle_pr_final',
 'run_name': 'baseline_model_logging',
 'registry_model_name': 'finproj_rang_models',
 'model_path': '/home/mle-user/mle_projects/mle-project-sprint-1-v001/part2_dvc/models/fitted_model.pkl',
 'data_path': '/home/mle-user/mle_projects/mle-project-sprint-1-v001/part2_dvc/data/initial_data.csv',
 'metrics_path': '/home/mle-user/mle_projects/mle-project-sprint-1-v001/part2_dvc/cv_results/cv_res.json',
 'requirements_path': '/home/mle-user/mle_projects/mle-dvc/requirements.txt',
 'mlflow_dir_path': '/home/mle-user/mle_projects/mle-pr-final/infrastructure'}

In [6]:
arch_path = '/home/mle-user/mle_projects/mle-pr-final/data/raw'
arch_name ='archive.zip'
dest = '/home/mle-user/mle_projects/mle-pr-final/data/raw'

In [7]:
data_path = '/home/mle-user/mle_projects/mle-pr-final/data/raw'

In [8]:
if len(os.listdir(data_path))==1:
     zip_path = os.path.join(arch_path, arch_name)
     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall(dest)
else:
     pass

In [9]:
os.listdir(data_path)

['category_tree.csv',
 'item_properties_part1.csv',
 'events.csv',
 'item_properties_part2.csv',
 'archive.zip']

In [10]:
category_tree = pd.read_csv(os.path.join(data_path, 'category_tree.csv'))
events = pd.read_csv(os.path.join(data_path, 'events.csv'))
item_properties_part1 = pd.read_csv(os.path.join(data_path, 'item_properties_part1.csv'))
item_properties_part2 = pd.read_csv(os.path.join(data_path, 'item_properties_part2.csv'))

In [11]:
def explore_data(df):
    """
    Исследует структуру данных: типы данных, пропущенные значения, базовые статистики.
    """
    print("===== Основная информация о данных =====")
    print(f"Размер датасета: {df.shape}")
    print("\nТипы данных:")
    print(df.dtypes)
    
    print("\nПропущенные значения:")
    missing_values = df.isnull().sum()
    print(missing_values[missing_values > 0])
    
    print("\nБазовая статистика:")
    print(df.describe())
    print(df.head(3))

In [12]:
explore_data(category_tree)

===== Основная информация о данных =====
Размер датасета: (1669, 2)

Типы данных:
categoryid      int64
parentid      float64
dtype: object

Пропущенные значения:
parentid    25
dtype: int64

Базовая статистика:
        categoryid     parentid
count  1669.000000  1644.000000
mean    849.285201   847.571168
std     490.195116   505.058485
min       0.000000     8.000000
25%     427.000000   381.000000
50%     848.000000   866.000000
75%    1273.000000  1291.000000
max    1698.000000  1698.000000
   categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0


In [13]:
explore_data(events)

===== Основная информация о данных =====
Размер датасета: (2756101, 5)

Типы данных:
timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object

Пропущенные значения:
transactionid    2733644
dtype: int64

Базовая статистика:
          timestamp     visitorid        itemid  transactionid
count  2.756101e+06  2.756101e+06  2.756101e+06   22457.000000
mean   1.436424e+12  7.019229e+05  2.349225e+05    8826.497796
std    3.366312e+09  4.056875e+05  1.341954e+05    5098.996290
min    1.430622e+12  0.000000e+00  3.000000e+00       0.000000
25%    1.433478e+12  3.505660e+05  1.181200e+05    4411.000000
50%    1.436453e+12  7.020600e+05  2.360670e+05    8813.000000
75%    1.439225e+12  1.053437e+06  3.507150e+05   13224.000000
max    1.442545e+12  1.407579e+06  4.668670e+05   17671.000000
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214

In [14]:
explore_data(item_properties_part1)

===== Основная информация о данных =====
Размер датасета: (10999999, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  1.100000e+07  1.100000e+07
mean   1.435158e+12  2.333851e+05
std    3.327653e+09  1.348258e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165150e+05
50%    1.433646e+12  2.334990e+05
75%    1.437880e+12  3.501860e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566


In [15]:
explore_data(item_properties_part2)

===== Основная информация о данных =====
Размер датасета: (9275903, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  9.275903e+06  9.275903e+06
mean   1.435156e+12  2.333968e+05
std    3.327970e+09  1.348682e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165175e+05
50%    1.433646e+12  2.334620e+05
75%    1.437880e+12  3.504470e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid property            value
0  1433041200000  183478      561           769062
1  1439694000000  132256      976  n26.400 1135780
2  1435460400000  420307      921  1149317 1257525


In [16]:
category_tree.categoryid.nunique(),category_tree.parentid.nunique(), category_tree.shape[0]

(1669, 362, 1669)

In [17]:
1669/362

4.610497237569061

### заметка

Количество категорий больше в 4,6 раза чем parentid, значит parentid это что то вроде группы, например как для треков был жанр.
Есть 26 категорий без parentid, присвоим им parentid равный 0, так как такого parentid в первоначальных данных нет.

In [18]:
events.head(3)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN


In [19]:
events.event.value_counts()

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

In [20]:
events.event.value_counts()/events.shape[0]

event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: count, dtype: float64

In [21]:
event_gr = events.groupby('event', as_index=False).agg({'visitorid': 'nunique', 'itemid': 'nunique'})

In [22]:
event_gr

,event,visitorid,itemid
0,addtocart,37722,23903
1,transaction,11719,12025
2,view,1404179,234838


In [23]:
event_gr['visitor_share'] = event_gr['visitorid'] / events['visitorid'].nunique()
event_gr['item_share'] = event_gr['itemid'] / events['itemid'].nunique()

In [24]:
event_gr

,event,visitorid,itemid,visitor_share,item_share
0,addtocart,37722,23903,0.026799,0.101688
1,transaction,11719,12025,0.008326,0.051157
2,view,1404179,234838,0.997584,0.999051


In [25]:
events[~events.transactionid.isna()].head(3)

,timestamp,visitorid,event,itemid,transactionid
130,1433222276276,599528,transaction,356475,4000.0
304,1433193500981,121688,transaction,15335,11117.0
418,1433193915008,552148,transaction,81345,5444.0


In [26]:
events[events.transactionid.isna()].shape[0], events.shape[0]

(2733644, 2756101)

In [27]:
item_properties_part1.head(3)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566


In [28]:
item_properties_part1[item_properties_part1['itemid'] == 460429]

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
2122725,1439694000000,460429,202,692224
2595719,1432436400000,460429,839,963847
6332062,1431226800000,460429,917,692224
6490575,1431226800000,460429,364,1058790
8129781,1431226800000,460429,available,0
8368211,1431226800000,460429,283,1103756 9705 963847
8566995,1433646000000,460429,283,1103756 9705 963847
8752572,1431226800000,460429,6,9705
8753668,1431226800000,460429,776,674847


In [29]:
item_properties_part2[item_properties_part2['itemid'] == 460429]

,timestamp,itemid,property,value
151187,1438484400000,460429,283,9705 963847 692224
349970,1442113200000,460429,283,9705 963847 692224
548753,1439694000000,460429,283,9705 963847 692224
563950,1431226800000,460429,888,692224
747536,1436065200000,460429,283,9705 963847 692224
788083,1435460400000,460429,764,1285872
1266667,1435460400000,460429,227,1103756 9705
1465450,1431226800000,460429,227,1103756 9705
1664233,1431831600000,460429,227,1103756 9705
1863016,1433041200000,460429,227,1103756 9705


In [30]:
item_properties_part1.property.value_counts(ascending=False)[0:10]

property
888           1629817
790            970800
available      817387
categoryid     426305
6              343207
283            323681
776            311654
678            261829
364            256340
202            242984
Name: count, dtype: int64

In [31]:
item_properties_part2.head(3)

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525


In [32]:
category_tree[category_tree['categoryid'] == 1338]

,categoryid,parentid
742,1338,1278.0


In [33]:
category_tree[category_tree['parentid'] == 1278.0]

,categoryid,parentid
170,578,1278.0
742,1338,1278.0
1134,1374,1278.0


In [34]:
category_tree.head(3)

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0


In [35]:
categoryid_prop1 = item_properties_part1[item_properties_part1['property'] == 'categoryid']
categoryid_prop2 = item_properties_part2[item_properties_part2['property'] == 'categoryid']
is_available1 = item_properties_part1[item_properties_part1['property'] == 'available']
is_available2 = item_properties_part1[item_properties_part1['property'] == 'available']

In [36]:
is_available = pd.concat([is_available1, is_available2])
category_prop = pd.concat([categoryid_prop1, categoryid_prop2])

### Признак доступности довара разделим на две таблицы, когда был доступен и когда нет.

In [37]:
is_available 

,timestamp,itemid,property,value
5,1436065200000,285026,available,0
15,1437274800000,186518,available,0
79,1433646000000,423682,available,0
82,1434250800000,316253,available,1
96,1437274800000,430459,available,0
...,...,...,...,...
10999926,1431831600000,350051,available,0
10999933,1433646000000,358669,available,0
10999959,1442113200000,362293,available,1
10999986,1439089200000,259404,available,1


In [38]:
is_available.groupby('value', as_index=False).agg({'itemid':'nunique','timestamp': 'count'})

,value,itemid,timestamp
0,0,219932,939226
1,1,57829,695548


In [39]:
is_available['value'] = is_available['value'].astype(int)

In [40]:
is_available_true = is_available[is_available['value'] ==1]
is_available_false = is_available[is_available['value'] ==0]

In [41]:
is_available.shape[0], is_available_true.shape[0]+ is_available_false.shape[0]

(1634774, 1634774)

In [42]:
is_available_true.columns = ['timestamp_available_true', 'itemid', 'property', 'value']
is_available_false.columns  = ['timestamp_available_false', 'itemid', 'property', 'value']

In [43]:
is_available_true['timestamp_available_true_min'] = is_available_true['timestamp_available_true']
is_available_false['timestamp_available_false_min'] = is_available_false['timestamp_available_false']

/tmp/ipykernel_4003/1894990614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_available_true['timestamp_available_true_min'] = is_available_true['timestamp_available_true']
/tmp/ipykernel_4003/1894990614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_available_false['timestamp_available_false_min'] = is_available_false['timestamp_available_false']


In [44]:
is_available_true_gr = is_available_true.groupby('itemid', as_index=False).agg({'timestamp_available_true': 'max', 'timestamp_available_true_min': 'min'})

In [45]:
is_available_false_gr = is_available_false.groupby('itemid', as_index=False).agg({'timestamp_available_false': 'max', 'timestamp_available_false_min': 'min'})

In [46]:
events

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [47]:
import psutil

print(psutil.virtual_memory())  # Информация о памяти
print(psutil.cpu_percent())  

svmem(total=33651257344, available=29274599424, percent=13.0, used=3885559808, free=26319384576, active=1709539328, inactive=4309442560, buffers=559546368, cached=2886766592, shared=1179648, slab=1181724672)
17.9


In [48]:
import gc
del item_properties_part1
del item_properties_part2
del is_available
gc.collect()

0

In [49]:
merged = events.merge(is_available_true_gr,
    how='left',on='itemid')

In [50]:
merged.shape[0], events.shape[0]

(2756101, 2756101)

In [51]:
merged = merged.merge(is_available_false_gr,how='left', on='itemid')

In [52]:
merged 

,timestamp,visitorid,event,itemid,transactionid,timestamp_available_true,timestamp_available_true_min,timestamp_available_false,timestamp_available_false_min
0,1433221332117,257597,view,355908,NaN,1.440904e+12,1.431832e+12,NaN,NaN
1,1433224214164,992329,view,248676,NaN,1.431227e+12,1.431227e+12,NaN,NaN
2,1433221999827,111016,view,318965,NaN,NaN,NaN,NaN,NaN
3,1433221955914,483717,view,253185,NaN,NaN,NaN,NaN,NaN
4,1433221337106,951259,view,367447,NaN,1.440904e+12,1.431227e+12,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,NaN,NaN,1.433041e+12,1.433041e+12
2756097,1438399813142,762376,view,115946,NaN,NaN,NaN,1.431227e+12,1.431227e+12
2756098,1438397820527,1251746,view,78144,NaN,1.442113e+12,1.431227e+12,1.441508e+12,1.440904e+12
2756099,1438398530703,1184451,view,283392,NaN,NaN,NaN,NaN,NaN


In [53]:
del events
del is_available_true 
del is_available_false
gc.collect()

0

In [54]:
category_tree['categoryid'] = category_tree['categoryid'].astype(str)

In [55]:
category_prop = category_prop.merge(category_tree, how='left', left_on='value', right_on='categoryid')

In [56]:
category_prop['parentid'] = category_prop['parentid'].fillna(0).astype(int)

In [57]:
category_prop_gr = category_prop.groupby('itemid', as_index=False).parentid.unique()

In [58]:
category_prop_gr

,itemid,parentid
0,0,[293]
1,1,[113]
2,2,[1214]
3,3,[938]
4,4,[1174]
...,...,...
417048,466862,[480]
417049,466863,[788]
417050,466864,[340]
417051,466865,[1424]


In [59]:
merged = merged.merge(category_prop_gr,
    how='left', on='itemid')

In [60]:
merged = merged.fillna(0)

In [61]:
cols_to_tr = ['timestamp', 'visitorid', 'itemid', 'transactionid',
       'timestamp_available_true', 'timestamp_available_true_min',
       'timestamp_available_false', 'timestamp_available_false_min']

In [62]:
for c in cols_to_tr:
    merged[c] = merged[c].astype(int)


In [84]:
merged[merged['transactionid']>0]['timestamp_available_true_data'].apply(lambda x: x.year)

AttributeError: 'Series' object has no attribute 'year'

In [67]:
merged.to_csv(os.path.join('/home/mle-user/mle_projects/mle-pr-final/data/processed', 'big_tab.csv.gz' ))

In [72]:
from datetime import datetime
def timestamp2date(timestamp):
    """Converts timestamp into a date."""
    date = datetime.fromtimestamp(timestamp / 1000)
    return date.date()

In [76]:
merged.head(1)

,timestamp,visitorid,event,itemid,transactionid,timestamp_available_true,timestamp_available_true_min,timestamp_available_false,timestamp_available_false_min,parentid,trans_data,timestamp_available_true_data,timestamp_available_true_min_data,timestamp_available_false_data,timestamp_available_false_min_data
0,1433221332117,257597,view,355908,0,1440903600000,1431831600000,0,0,[805],2015-06-02,2015-08-30,2015-05-17,1970-01-01,1970-01-01


In [73]:
merged['trans_data'] = merged['timestamp'].apply(timestamp2date)

In [75]:
merged['timestamp_available_true_data'] = merged['timestamp_available_true'].apply(timestamp2date)
merged['timestamp_available_true_min_data'] = merged['timestamp_available_true_min'].apply(timestamp2date)
merged['timestamp_available_false_data'] = merged['timestamp_available_false'].apply(timestamp2date)
merged['timestamp_available_false_min_data'] = merged['timestamp_available_false_min'].apply(timestamp2date)

In [85]:
merged['is_av_y'] = merged['timestamp_available_true_data'].apply(lambda x: x.year)

In [89]:
merged.groupby('event', as_index=0)['is_av_y'].value_counts()

,event,is_av_y,count
0,addtocart,2015,43143
1,addtocart,1970,26189
2,transaction,2015,13693
3,transaction,1970,8764
4,view,1970,1629548
5,view,2015,1034764


In [90]:
merged.head(1)

,timestamp,visitorid,event,itemid,transactionid,timestamp_available_true,timestamp_available_true_min,timestamp_available_false,timestamp_available_false_min,parentid,trans_data,timestamp_available_true_data,timestamp_available_true_min_data,timestamp_available_false_data,timestamp_available_false_min_data,is_av_y
0,1433221332117,257597,view,355908,0,1440903600000,1431831600000,0,0,[805],2015-06-02,2015-08-30,2015-05-17,1970-01-01,1970-01-01,2015


In [92]:
merged['parentid']=merged['parentid'].astype(str)

In [94]:
category_events = merged.groupby(['event','parentid'], as_index=0).agg({'itemid':'nunique','timestamp': 'count'})

In [102]:
category_events[category_events['event']=="view"].sort_values('timestamp', ascending=0)[0:30] 

,event,parentid,itemid,timestamp
779,view,0,49814,254277
1189,view,[1606],3501,101425
1470,view,[955],3128,94464
1342,view,[561],4214,91351
1332,view,[500],6882,83121
1119,view,[1426],3263,67920
1421,view,[805],2030,56101
968,view,[1095],1975,52686
1151,view,[1497],2945,50365
1255,view,[250 679],1908,46448


In [101]:
category_events[category_events['event']=='addtocart'].sort_values('timestamp', ascending=0)[0:30]

,event,parentid,itemid,timestamp
331,addtocart,[561],562,2645
418,addtocart,[955],499,2250
191,addtocart,[1426],482,2200
239,addtocart,[1606],567,2159
325,addtocart,[500],560,2134
98,addtocart,[1095],341,1693
162,addtocart,[1308],478,1508
383,addtocart,[805],407,1410
88,addtocart,[105],659,1362
276,addtocart,[250 679],199,1191


In [103]:
category_events[category_events['event']=='transaction'].sort_values('timestamp', ascending=0)[0:30] 

,event,parentid,itemid,timestamp
696,transaction,[561],331,880
773,transaction,[955],275,667
499,transaction,[1095],223,556
742,transaction,[805],252,553
492,transaction,[105],352,552
690,transaction,[500],262,531
577,transaction,[1426],196,489
425,transaction,0,380,475
553,transaction,[1308],241,449
618,transaction,[1606],227,434


In [98]:
category_events['event'].unique()

array(['addtocart', 'transaction', 'view'], dtype=object)